In [1]:
import tensorflow as tf

# Set paths for train, validation, and test datasets
train_dataset_path = 'Deepfake_dataset/Train'
validation_dataset_path = 'Deepfake_dataset/Validation'
test_dataset_path = 'Deepfake_dataset/Test'

img_size = (256, 256)
batch_size = 32

# Load the training dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dataset_path,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical',  # 'categorical' for multi-class, 'binary' for binary classification
    shuffle=True
)

# Load the validation dataset
validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    validation_dataset_path,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical',
    shuffle=False  # No need to shuffle for validation
)

# Load the test dataset (used later for evaluation)
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dataset_path,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical',
    shuffle=False
)

Found 140002 files belonging to 2 classes.
Found 39428 files belonging to 2 classes.
Found 10905 files belonging to 2 classes.


In [2]:
from tensorflow.keras.applications import ResNet50 # type: ignore
from tensorflow.keras import layers, models # type: ignore

# Load the pretrained ResNet50 model (without the top fully connected layers)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze the base model's layers to use its pretrained features
base_model.trainable = False

# Build the model by adding a global pooling layer and a classification head
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),  # You can adjust the size of the dense layer
    layers.Dense(2, activation='softmax')  # 2 classes: real and fake (adjust to your dataset)
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 8, 8, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,850,242 (90.98 MB)

 Trainable params: 262,530 (1.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
# Set the number of epochs
epochs = 1  # Adjust according to your needs

# Train the model
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=epochs
)

1100/4376 ━━━━━━━━━━━━━━━━━━━━ 3:18:22 4s/step - accuracy: 0.7895 - loss: 0.4413

In [ ]:
# Evaluate on the test dataset
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

In [ ]:
import matplotlib.pyplot as plt

# Plot accuracy curves
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()

# Plot loss curves
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()

In [ ]:
model_json=model.to_json()
with open("Resnet_model.json",'w') as json_file:
     json_file.write(model_json)

In [ ]:
model.save_weights('Resnet_weights.weights.h5')